### Complex Schema Validation with Avro
**Description**: Implement a solution in Python to validate records against a complex nested Avro schema.

Eg., Complex schema ( nested_schema.avsc ):

**Steps**:
1. Load schema
2. Example data to validate
3. Validate against schema
4. Read back to check

In [1]:
# write your code from here
!pip install fastavro


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 8.6 MB/s eta 0:00:00:00:010:01

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import json
from fastavro import parse_schema, writer, reader, validation
import io

# Step 1: Load schema
with open("/workspaces/AI_DATA_ANALYSIS_/src/Module 9/Data Quality Challenges in Data Engineering Pipelines/nested_schema.avsc", "r") as f:
    schema = json.load(f)
parsed_schema = parse_schema(schema)

# Step 2: Example data
valid_record = {
    "username": "johndoe",
    "age": 30,
    "address": {
        "street": "123 Main St",
        "city": "Metropolis",
        "zip": 12345
    },
    "preferences": {
        "newsletter": True,
        "notifications": ["email", "sms"]
    }
}

invalid_record = {
    "username": "janedoe",
    "age": "thirty",  # Invalid type
    "address": {
        "street": "456 Side St",
        "city": "Gotham",
        "zip": "abcde"  # Invalid type
    },
    "preferences": {
        "newsletter": "yes",  # Should be boolean
        "notifications": ["push"]
    }
}

# Step 3: Validate records
def validate_record(record, schema):
    try:
        validation.validate(record, schema)
        return True
    except validation.ValidationError as e:
        print(f"Validation error: {e}")
        return False

print("✅ Validating valid record:")
print(validate_record(valid_record, parsed_schema))

print("\n❌ Validating invalid record:")
print(validate_record(invalid_record, parsed_schema))

# Step 4: Write to in-memory buffer and read back to check
buffer = io.BytesIO()
writer(buffer, parsed_schema, [valid_record])
buffer.seek(0)

print("\n📦 Reading back record:")
for rec in reader(buffer):
    print(rec)


✅ Validating valid record:
True

❌ Validating invalid record:
Validation error: [
  "User.age is <thirty> of type <class 'str'> expected int"
]
False

📦 Reading back record:
{'username': 'johndoe', 'age': 30, 'address': {'street': '123 Main St', 'city': 'Metropolis', 'zip': 12345}, 'preferences': {'newsletter': True, 'notifications': ['email', 'sms']}}
